In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import m2cgen as m2c
import joblib
import pickle
import glob
import sys
from sklearn.model_selection import GridSearchCV
import m2cgen
import math


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
print("done")



In [ ]:
# Import data to use as training data

dataframes_in = []
dataframes_out = []
files = glob.glob("clean_csv/*.csv")
print(files)
for filename in files:
    df = pd.read_csv(filename,  index_col=False)
    print(filename)
    print(filename[10])
    if filename[10] == "i":

        dataframes_in.append(df)
    else:
        dataframes_out.append(df)

in_data = pd.concat(dataframes_in)
out_data = pd.concat(dataframes_out)

# Outdoors is positive
print(in_data.shape)
print(out_data.shape)
in_data["target"] = 0
out_data["target"] = 1

all_data = pd.concat([in_data, out_data])


In [ ]:

all_data['accelN'] = all_data.apply(lambda row: (row.accelX + row.accelY + row.accelZ) / 3, axis=1)
all_data['accelMagnitude'] = all_data.apply(lambda row: row.accelX ** 2 + row.accelY ** 2 + row.accelZ ** 2, axis=1)
all_data['accelXY'] = all_data.apply(lambda row: abs(row.accelX * row.accelY), axis=1)


# # rgb vs clear
all_data['red/clear'] = all_data.apply(lambda row: row.r / (row.c + 1), axis=1)
all_data['green/clear'] = all_data.apply(lambda row: row.g /  (row.c + 1), axis=1)
all_data['blue/clear'] = all_data.apply(lambda row: row.b /  (row.c + 1), axis=1)

# # rgb versus each other
all_data['red/green'] = all_data.apply(lambda row: row.r / (row.g + 1), axis=1)
all_data['blue/red'] = all_data.apply(lambda row: row.b / (row.r + 1), axis=1)
all_data['blue/green'] = all_data.apply(lambda row: row.b / (row.g + 1), axis=1)

# relative difference of each colour compared to rgbc channels
all_data["(clear-red)/red"] = all_data.apply(lambda row: (row.c - row.r) / (row.r + 1), axis=1)
all_data["(clear-green)/red"] = all_data.apply(lambda row: (row.c - row.g) / (row.r + 1), axis=1)
all_data["(clear-blue)/red"] = all_data.apply(lambda row: (row.c - row.b) / (row.r + 1), axis=1)

all_data["(clear-red)/green"] = all_data.apply(lambda row: (row.c - row.r) / (row.g + 1), axis=1)
all_data["(clear-green)/green"] = all_data.apply(lambda row: (row.c - row.g) / (row.g + 1), axis=1)
all_data["(clear-blue)/green"] = all_data.apply(lambda row: (row.c - row.b) / (row.g + 1), axis=1)

all_data["(clear-red)/blue"] = all_data.apply(lambda row: (row.c - row.r) / (row.b + 1), axis=1)
all_data["(clear-green)/blue"] = all_data.apply(lambda row: (row.c - row.g) / (row.b + 1), axis=1)
all_data["(clear-blue)/blue"] = all_data.apply(lambda row: (row.c - row.b) / (row.b + 1), axis=1)

all_data["(clear-red)/clear"] = all_data.apply(lambda row: (row.c - row.r) / (row.c + 1), axis=1)
all_data["(clear-green)/clear"] = all_data.apply(lambda row: (row.c - row.g) / (row.c + 1), axis=1)
all_data["(clear-blue)/clear"] = all_data.apply(lambda row: (row.c - row.b) / (row.c + 1), axis=1)

# Okay until log

# log(rgb) vs log(clear)
all_data['log(red)/log(clear)'] = all_data.apply(lambda row: math.log(row.r + 1)/ math.log(row.c + 1), axis=1)
all_data['log(green)/log(clear)'] = all_data.apply(lambda row: math.log(row.g + 1) / math.log(row.c + 1), axis=1)
all_data['log(blue)/log(clear)'] = all_data.apply(lambda row: math.log(row.b + 1) / math.log(row.c + 1), axis=1)



#log(rgb) versus each other
all_data['log(red)/log(green)'] = all_data.apply(lambda row: math.log(row.r + 1) / math.log(row.g + 1), axis=1)
all_data['log(blue)/log(red)'] = all_data.apply(lambda row: math.log(row.b + 1) / math.log(row.r + 1), axis=1)
all_data['log(blue)/log(green)'] = all_data.apply(lambda row: math.log(row.b + 1) / math.log(row.g + 1), axis=1)

# sqrt(rgb) vs sqrt(clear)
all_data['sqrt(red)/sqrt(clear)'] = all_data.apply(lambda row: math.sqrt(row.r + 1)/ math.sqrt(row.c + 1), axis=1)
all_data['sqrt(green)/sqrt(clear)'] = all_data.apply(lambda row: math.sqrt(row.g + 1) / math.sqrt(row.c + 1), axis=1)
all_data['sqrt(blue)/sqrt(clear)'] = all_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.c + 1), axis=1)

#sqrt(rgb) vs each other
all_data['sqrt(red)/sqrt(green)'] = all_data.apply(lambda row: math.sqrt(row.r + 1) / math.sqrt(row.g + 1), axis=1)
all_data['sqrt(blue)/sqrt(red)'] = all_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.r + 1), axis=1)
all_data['sqrt(blue)/sqrt(green)'] = all_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.g + 1), axis=1)


# # uv vs lux
all_data['lux/uv'] = all_data.apply(lambda row: row.lux / (row.uv + 1), axis=1) 
all_data['blue/uv'] = all_data.apply(lambda row: row.b / (row.uv + 1), axis=1) 
all_data['(clear - blue)/uv'] = all_data.apply(lambda row: (row.c - row.b) / (row.uv + 1), axis=1)
all_data['log(lux)/log(uv)'] = all_data.apply(lambda row:  math.log(row.lux + 1)/ math.log(row.uv + 2), axis=1)
all_data['log(blue)/log(uv)'] = all_data.apply(lambda row:  math.log(row.b + 1)/ math.log(row.uv + 2), axis=1)
all_data['sqrt(lux)/sqrt(uv)'] = all_data.apply(lambda row:  math.sqrt(row.lux + 1)/ math.sqrt(row.uv + 1), axis=1)
all_data['sqrt(blue)/sqrt(uv)'] = all_data.apply(lambda row:  math.sqrt(row.b + 1)/ math.sqrt(row.uv + 1), axis=1)


X_train = all_data.drop(['timestamp', 'target'], axis=1).copy()
y_train = all_data['target'].copy()
column_names = list(X_train.columns)
print(column_names)

In [ ]:
#'boosting_type': 'dart', 'min_data_in_leaf': 400, 'num_leaves': 31, 'reg_alpha': 0.1}
lgbm_param_grid = {
    'num_leaves': [20, 31, 80],
    'min_child_samples': [ 20, 50, 100],
    'max_depth': [10, 50, 100, 200],
    "boosting_type": ['gbdt', 'dart']
    }

lgb_model = LGBMClassifier(random_state = 42)
lgb_grid_search = GridSearchCV(lgb_model, lgbm_param_grid, scoring='roc_auc', cv=5)
lgb_grid_search.fit(X_train, y_train) 

In [ ]:

print("Best set of hyperparameters: ", lgb_grid_search.best_params_)
print("Best score: ", lgb_grid_search.best_score_)

feature_importances = lgb_grid_search.best_estimator_.feature_importances_

fig = plt.figure(figsize=(12,6))
plt.bar(column_names, feature_importances)
plt.xticks(rotation=90)
plt.title("LightGBM Feature Importance")
plt.grid()
plt.show()

for feature, importance in zip(column_names, feature_importances):
    if importance > 0.002:
        print(f"{feature}: {importance:.3f}")

In [ ]:
# classifier = lgb_grid_search.best_estimator_

classifier = LGBMClassifier(random_state = 42, boosting_type="'gbdt'", reg_alpha=0, reg_lambda=0, min_child_samples=20, num_leaves=20, max_depth=50)
classifier.fit(X_train, y_train)
with open('light_gbm_small02.dart', 'w') as f:
    code = m2cgen.export_to_dart(classifier)
    f.write(code)



Best set of hyperparameters:  {'boosting_type': 'gbdt', 'max_depth': 50, 'min_child_samples': 20, 'num_leaves': 20}
Best score:  0.9812264236310542

7814 lines


Best set of hyperparameters:  {'boosting_type': 'gbdt', 'max_depth': 50, 'min_child_samples': 20, 'num_leaves': 20}
Best score:  0.9812264236310542

In [ ]:

valid_df = []

files = glob.glob("validation_csv/*.csv")
print(files)
for filename in files:
    df = pd.read_csv(filename,  index_col=False)
    print(filename)
    print(df[df["uv"]< 0])
    valid_df.append(df)


val_data = pd.concat(valid_df)


# Outdoors is positive
print(val_data.shape)


import math
# # accelerometer

val_data['accelN'] = val_data.apply(lambda row: (row.accelX + row.accelY + row.accelZ) / 3, axis=1)
val_data['accelMagnitude'] = val_data.apply(lambda row: row.accelX ** 2 + row.accelY ** 2 + row.accelZ ** 2, axis=1)
val_data['accelXY'] = val_data.apply(lambda row: abs(row.accelX * row.accelY), axis=1)


# # rgb vs clear
val_data['red/clear'] = val_data.apply(lambda row: row.r / (row.c + 1), axis=1)
val_data['green/clear'] = val_data.apply(lambda row: row.g /  (row.c + 1), axis=1)
val_data['blue/clear'] = val_data.apply(lambda row: row.b /  (row.c + 1), axis=1)

# # rgb versus each other
val_data['red/green'] = val_data.apply(lambda row: row.r / (row.g + 1), axis=1)
val_data['blue/red'] = val_data.apply(lambda row: row.b / (row.r + 1), axis=1)
val_data['blue/green'] = val_data.apply(lambda row: row.b / (row.g + 1), axis=1)

# relative difference of each colour compared to rgbc channels
val_data["(clear-red)/red"] = val_data.apply(lambda row: (row.c - row.r) / (row.r + 1), axis=1)
val_data["(clear-green)/red"] = val_data.apply(lambda row: (row.c - row.g) / (row.r + 1), axis=1)
val_data["(clear-blue)/red"] = val_data.apply(lambda row: (row.c - row.b) / (row.r + 1), axis=1)

val_data["(clear-red)/green"] = val_data.apply(lambda row: (row.c - row.r) / (row.g + 1), axis=1)
val_data["(clear-green)/green"] = val_data.apply(lambda row: (row.c - row.g) / (row.g + 1), axis=1)
val_data["(clear-blue)/green"] = val_data.apply(lambda row: (row.c - row.b) / (row.g + 1), axis=1)

val_data["(clear-red)/blue"] = val_data.apply(lambda row: (row.c - row.r) / (row.b + 1), axis=1)
val_data["(clear-green)/blue"] = val_data.apply(lambda row: (row.c - row.g) / (row.b + 1), axis=1)
val_data["(clear-blue)/blue"] = val_data.apply(lambda row: (row.c - row.b) / (row.b + 1), axis=1)

val_data["(clear-red)/clear"] = val_data.apply(lambda row: (row.c - row.r) / (row.c + 1), axis=1)
val_data["(clear-green)/clear"] = val_data.apply(lambda row: (row.c - row.g) / (row.c + 1), axis=1)
val_data["(clear-blue)/clear"] = val_data.apply(lambda row: (row.c - row.b) / (row.c + 1), axis=1)

# Okay until log

# log(rgb) vs log(clear)
val_data['log(red)/log(clear)'] = val_data.apply(lambda row: math.log(row.r + 1)/ math.log(row.c + 1), axis=1)
val_data['log(green)/log(clear)'] = val_data.apply(lambda row: math.log(row.g + 1) / math.log(row.c + 1), axis=1)
val_data['log(blue)/log(clear)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.c + 1), axis=1)



#log(rgb) versus each other
val_data['log(red)/log(green)'] = val_data.apply(lambda row: math.log(row.r + 1) / math.log(row.g + 1), axis=1)
val_data['log(blue)/log(red)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.r + 1), axis=1)
val_data['log(blue)/log(green)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.g + 1), axis=1)

# sqrt(rgb) vs sqrt(clear)
val_data['sqrt(red)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.r + 1)/ math.sqrt(row.c + 1), axis=1)
val_data['sqrt(green)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.g + 1) / math.sqrt(row.c + 1), axis=1)
val_data['sqrt(blue)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.c + 1), axis=1)

#sqrt(rgb) vs each other
val_data['sqrt(red)/sqrt(green)'] = val_data.apply(lambda row: math.sqrt(row.r + 1) / math.sqrt(row.g + 1), axis=1)
val_data['sqrt(blue)/sqrt(red)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.r + 1), axis=1)
val_data['sqrt(blue)/sqrt(green)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.g + 1), axis=1)


# # uv vs lux
val_data['lux/uv'] = val_data.apply(lambda row: row.lux / (row.uv + 1), axis=1) 
val_data['blue/uv'] = val_data.apply(lambda row: row.b / (row.uv + 1), axis=1) 
val_data['(clear - blue)/uv'] = val_data.apply(lambda row: (row.c - row.b) / (row.uv + 1), axis=1)
val_data['log(lux)/log(uv)'] = val_data.apply(lambda row:  math.log(row.lux + 1)/ math.log(row.uv + 2), axis=1)
val_data['log(blue)/log(uv)'] = val_data.apply(lambda row:  math.log(row.b + 1)/ math.log(row.uv + 2), axis=1)
val_data['sqrt(lux)/sqrt(uv)'] = val_data.apply(lambda row:  math.sqrt(row.lux + 1)/ math.sqrt(row.uv + 1), axis=1)
val_data['sqrt(blue)/sqrt(uv)'] = val_data.apply(lambda row:  math.sqrt(row.b + 1)/ math.sqrt(row.uv + 1), axis=1)


X_val = val_data.drop(['timestamp', 'target'], axis=1).copy()
y_val = val_data['target'].copy()
column_names = list(X_val.columns)
print(column_names)



In [ ]:
y_pred_probs = classifier.predict_proba(X_val)[:, 1]
fpr, tpr, _ = roc_curve(y_val, y_pred_probs)
auc = roc_auc_score(y_val, y_pred_probs)
print(auc)
plt.plot(fpr, tpr, label = f"LGBM, AUC = {auc:.3f}")
plt.legend()

In [ ]:

valid_df = []

files = glob.glob("validation_csv/*.csv")
print(files)
for filename in files:
    df = pd.read_csv(filename,  index_col=False)
    print(filename)
    print(df[df["uv"]< 0])
    valid_df.append(df)


val_data = pd.concat(valid_df)


# Outdoors is positive
print(val_data.shape)


import math
# # accelerometer

val_data['accelN'] = val_data.apply(lambda row: (row.accelX + row.accelY + row.accelZ) / 3, axis=1)
val_data['accelMagnitude'] = val_data.apply(lambda row: row.accelX ** 2 + row.accelY ** 2 + row.accelZ ** 2, axis=1)
val_data['accelXY'] = val_data.apply(lambda row: abs(row.accelX * row.accelY), axis=1)


# # rgb vs clear
val_data['red/clear'] = val_data.apply(lambda row: row.r / (row.c + 1), axis=1)
val_data['green/clear'] = val_data.apply(lambda row: row.g /  (row.c + 1), axis=1)
val_data['blue/clear'] = val_data.apply(lambda row: row.b /  (row.c + 1), axis=1)

# # rgb versus each other
val_data['red/green'] = val_data.apply(lambda row: row.r / (row.g + 1), axis=1)
val_data['blue/red'] = val_data.apply(lambda row: row.b / (row.r + 1), axis=1)
val_data['blue/green'] = val_data.apply(lambda row: row.b / (row.g + 1), axis=1)

# relative difference of each colour compared to rgbc channels
val_data["(clear-red)/red"] = val_data.apply(lambda row: (row.c - row.r) / (row.r + 1), axis=1)
val_data["(clear-green)/red"] = val_data.apply(lambda row: (row.c - row.g) / (row.r + 1), axis=1)
val_data["(clear-blue)/red"] = val_data.apply(lambda row: (row.c - row.b) / (row.r + 1), axis=1)

val_data["(clear-red)/green"] = val_data.apply(lambda row: (row.c - row.r) / (row.g + 1), axis=1)
val_data["(clear-green)/green"] = val_data.apply(lambda row: (row.c - row.g) / (row.g + 1), axis=1)
val_data["(clear-blue)/green"] = val_data.apply(lambda row: (row.c - row.b) / (row.g + 1), axis=1)

val_data["(clear-red)/blue"] = val_data.apply(lambda row: (row.c - row.r) / (row.b + 1), axis=1)
val_data["(clear-green)/blue"] = val_data.apply(lambda row: (row.c - row.g) / (row.b + 1), axis=1)
val_data["(clear-blue)/blue"] = val_data.apply(lambda row: (row.c - row.b) / (row.b + 1), axis=1)

val_data["(clear-red)/clear"] = val_data.apply(lambda row: (row.c - row.r) / (row.c + 1), axis=1)
val_data["(clear-green)/clear"] = val_data.apply(lambda row: (row.c - row.g) / (row.c + 1), axis=1)
val_data["(clear-blue)/clear"] = val_data.apply(lambda row: (row.c - row.b) / (row.c + 1), axis=1)

# Okay until log

# log(rgb) vs log(clear)
val_data['log(red)/log(clear)'] = val_data.apply(lambda row: math.log(row.r + 1)/ math.log(row.c + 1), axis=1)
val_data['log(green)/log(clear)'] = val_data.apply(lambda row: math.log(row.g + 1) / math.log(row.c + 1), axis=1)
val_data['log(blue)/log(clear)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.c + 1), axis=1)



#log(rgb) versus each other
val_data['log(red)/log(green)'] = val_data.apply(lambda row: math.log(row.r + 1) / math.log(row.g + 1), axis=1)
val_data['log(blue)/log(red)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.r + 1), axis=1)
val_data['log(blue)/log(green)'] = val_data.apply(lambda row: math.log(row.b + 1) / math.log(row.g + 1), axis=1)

# sqrt(rgb) vs sqrt(clear)
val_data['sqrt(red)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.r + 1)/ math.sqrt(row.c + 1), axis=1)
val_data['sqrt(green)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.g + 1) / math.sqrt(row.c + 1), axis=1)
val_data['sqrt(blue)/sqrt(clear)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.c + 1), axis=1)

#sqrt(rgb) vs each other
val_data['sqrt(red)/sqrt(green)'] = val_data.apply(lambda row: math.sqrt(row.r + 1) / math.sqrt(row.g + 1), axis=1)
val_data['sqrt(blue)/sqrt(red)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.r + 1), axis=1)
val_data['sqrt(blue)/sqrt(green)'] = val_data.apply(lambda row: math.sqrt(row.b + 1) / math.sqrt(row.g + 1), axis=1)


# # uv vs lux
val_data['lux/uv'] = val_data.apply(lambda row: row.lux / (row.uv + 1), axis=1) 
val_data['blue/uv'] = val_data.apply(lambda row: row.b / (row.uv + 1), axis=1) 
val_data['(clear - blue)/uv'] = val_data.apply(lambda row: (row.c - row.b) / (row.uv + 1), axis=1)
val_data['log(lux)/log(uv)'] = val_data.apply(lambda row:  math.log(row.lux + 1)/ math.log(row.uv + 2), axis=1)
val_data['log(blue)/log(uv)'] = val_data.apply(lambda row:  math.log(row.b + 1)/ math.log(row.uv + 2), axis=1)
val_data['sqrt(lux)/sqrt(uv)'] = val_data.apply(lambda row:  math.sqrt(row.lux + 1)/ math.sqrt(row.uv + 1), axis=1)
val_data['sqrt(blue)/sqrt(uv)'] = val_data.apply(lambda row:  math.sqrt(row.b + 1)/ math.sqrt(row.uv + 1), axis=1)


X_val = val_data.drop(['timestamp', 'target'], axis=1).copy()
y_val = val_data['target'].copy()
column_names = list(X_val.columns)
print(column_names)



In [ ]:
# J Youden index
y_pred = classifier.predict(X_val)
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
idx = np.argmax(tpr - fpr)
print( thresholds[idx])
auc = roc_auc_score(y_val, y_pred_probs)
print(auc)
cm = confusion_matrix(y_val, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[0,1])
disp.plot()
plt.show()